### 1. Import libraries

In [6]:
from ezyrb import POD, AE, GPR, ANN, RBF, Database
from ezyrb import ReducedOrderModel as ROM
import torch
import torch.nn as nn

from scipy.interpolate import RBFInterpolator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
np.bool = np.bool_ # to avoid error in pyvista

import matplotlib.pyplot as plt

from varname import nameof
import pyvista as pv

### 2. Read and preprocess of the data

In [2]:
# Assuming your data preparation steps are done here
num_simulations = 200
train_ratio = 0.8

# Load npy data from training_data folder
pressure_data = np.load('training_data/pressure_data_concat.npy')
velocity_data = np.load('training_data/velocity_data_concat.npy')
points_data = np.load('training_data/points_data.npy')

# Load design parameters from training_data folder
design_parameters = np.load('training_data/displacement_data_concat.npy')

In [3]:
random_state = 42

# randomly spit the data into training and testing data
train_pressure, test_pressure = train_test_split(pressure_data, train_size=train_ratio, random_state=random_state)
train_velocity, test_velocity = train_test_split(velocity_data, train_size=train_ratio, random_state=random_state)
train_design_parameters, test_design_parameters = train_test_split(design_parameters, train_size=train_ratio, random_state=random_state)
train_points, test_points = train_test_split(points_data, train_size=train_ratio, random_state=random_state)

In [4]:
# delete columns with all zeros in train_design_parameters
eliminate_columns = np.all(train_design_parameters == 0, axis=0)
train_design_parameters = train_design_parameters[:, ~eliminate_columns]
test_design_parameters = test_design_parameters[:, ~eliminate_columns]

### 3. AE evaluation

In [5]:
def PODI_rom(ae, model, train_design_parameters, train_data):
    # Define the database
    db = Database(train_design_parameters, train_data)
    # Define the ROM
    rom = ROM(db, ae, model)
    rom.fit()

    return rom

### 4. RBF interpolation

In [9]:
# Evaluate AE on the pressure data
low_dim = 20
optim = torch.optim.Adam
encoder_list = [2000, 200, low_dim]
decoder_list = [low_dim, 200, 2000]
encoder_activation = torch.nn.Tanh
decoder_activation = torch.nn.Tanh

p_ae = AE(encoder_list, decoder_list, encoder_activation(), decoder_activation(), 
          [2000, 1e-5], optimizer=optim, frequency_print=50)

# Define the ANN model
p_ann = ann = ANN([5, 20], nn.Tanh(), [2000,1e-5])

# Perform PODI on the pressure data
p_rom = PODI_rom(p_ae, p_ann, train_design_parameters, train_pressure)

# Predict the pressure data
p_predict = p_rom.predict(test_design_parameters)

# calculate the reconstrcuted error for each test data using mean squared error
p_reconstructed_error = np.linalg.norm(test_pressure - p_predict.snapshots_matrix, axis=1) / np.linalg.norm(test_pressure, axis=1)

# plot the error
plt.bar([i for i in range(len(p_reconstructed_error))], p_reconstructed_error)
plt.title('Pressure reconstructed error')
plt.xlabel('Test data')
plt.ylabel('Reconstructed error')
plt.show()

[epoch      1]	2.042527e-01
[epoch     50]	6.480517e-04


In [24]:
# Evaluate AE on the pressure data
low_dim = 50
optim = torch.optim.Adam
encoder_list = [2000, 100, low_dim]
decoder_list = [low_dim, 1000, 2000]
encoder_activation = torch.nn.Tanh
decoder_activation = torch.nn.Tanh

u_ae = AE(encoder_list, decoder_list, encoder_activation(), decoder_activation(), 
          [1000, 1e-5], optimizer=optim, frequency_print=20)

# Define the ANN model
u_ann = ANN([5, 20], nn.Tanh(), [2000,1e-5])

# Perform PODI on the pressure data
u_rom = PODI_rom(u_ae, u_ann, train_design_parameters, train_velocity)

# Predict the pressure data
u_predict = u_rom.predict(test_design_parameters)

# calculate the reconstrcuted error for each test data using mean squared error
u_reconstructed_error = np.linalg.norm(test_velocity - u_predict.snapshots_matrix, axis=1) / np.linalg.norm(test_velocity, axis=1)

# plot the error
plt.bar([i for i in range(len(u_reconstructed_error))], u_reconstructed_error)
plt.title('Velocity reconstructed error')
plt.xlabel('Test data')
plt.ylabel('Reconstructed error')
plt.show()

[epoch      1]	1.844543e-01
[epoch     20]	5.797183e-03
[epoch     40]	1.995192e-03
[epoch     60]	1.475034e-03
[epoch     80]	1.406138e-03


### 5. VTK visualization

In [ ]:
# Define a funtion to write the data into a VTK file
def vtk_writer(field_data, field_name, data_type,
               refVTM, save_path_name, points_data = None):
    # Add velocity data to each block within the MultiBlock dataset
    
    for block_i in range(refVTM.n_blocks):
        block = refVTM[block_i]
        if block is not None:
            if data_type == 'scalar':
                for data_i in range(len(field_name)):
                    block.cell_data[field_name[data_i]] = field_data[data_i]
            elif data_type == 'vector':
                for data_i in range(len(field_name)):
                    field = field_data[data_i].reshape(3, -1).T
                    block.cell_data[field_name[data_i]] = field 
            if points_data is not None:
                points = points_data.reshape(3, -1).T
                block.points = points

    # Save the modified VTM file
    output_vtm_file_path = f'{save_path_name}.vtm'
    refVTM.save(output_vtm_file_path)

In [18]:
# write the truth data, reconstructed data, and error into a VTK file
# Load the reference mesh VTM file
refVTM = pv.MultiBlock('simulation_data/refCase/VTK/refCase_0.vtm')
field_name = ['truth', 'reconstructed', 'error']
p_error_field = test_pressure - p_predict.snapshots_matrix

# loop all test data and write the data into VTK file
for i in range(len(test_pressure)):
    vtk_writer([test_pressure[i], p_predict.snapshots_matrix[i], p_error_field[i]], 
                field_name, 'scalar', refVTM, f'visualization_data/EZyRB_PODI_GPR/test_case_{i}_pressure', test_points[i])

In [19]:
# Write the velocity data into VTK file
field_name = ['truth', 'reconstructed', 'error']
u_error_field = test_velocity - u_predict.snapshots_matrix

# loop all test data and write the data into VTK file
for i in range(len(test_velocity)):
    vtk_writer([test_velocity[i], u_predict.snapshots_matrix[i], u_error_field[i]], 
                field_name, 'vector', refVTM, f'visualization_data/EZyRB_PODI_GPR/test_case_{i}_velocity', test_points[i])